In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
import pandas as pd

In [ ]:
from utils.addr_parser import AddressParser
from utils.addr_splitter import AddressSplitter

from utils.hepler_poi import corrections, company_name, token
from utils.hepler_poi import csv_extractor, convert_columns, apply_manual_corrections, process_dataframe_addresses
from utils.hepler_poi import enrich_chain_status, split_chain_company_name, fill_company_and_branch_by_id, geocode_tdx_df, add_catering_type_column, patch_coordinates

In [ ]:
# 全國營業(稅籍)登記資料集原始資料
csv_path = r"C:\labs\geo-grid\data\raw\National_Business_Registration_Dataset_202510.csv"
extracted =csv_extractor(csv_path)
print(f" 逐筆交易資料載入成功: {extracted.shape}  記憶體使用: {extracted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
df = extracted.loc[extracted['統一編號'].notna(),
    ['營業地址', '統一編號', '總機構統一編號', '營業人名稱', '資本額', '設立日期', '組織別名稱', '使用統一發票', '行業代號', '名稱']
].copy()

In [ ]:
df = convert_columns(df)

In [ ]:
splitter = AddressSplitter(
    input_column='營業地址',
    output_column='正規化營業地址',
    status_column='地址狀態',
    delimiters=None,             # 使用預設分隔符
    batch_size=10000,
    print_report=True,
    preprocessor=None          # 預設不做任何前處理
)
df_proc, changes = splitter.process(df)

In [ ]:
parser = AddressParser()
df_proc = parser.process(df_proc)

In [ ]:
# 資料補正
df_proc = apply_manual_corrections(
    df=df_proc,
    corrections=corrections
)

In [ ]:
# 篩選餐飲業poi
catering_services = df_proc.loc[
    # 1. 篩選特定縣市
    (df_proc['縣市'].isin(['臺北市', '新北市', '桃園市', '臺中市', '臺南市', '高雄市', '新竹市', '新竹縣'])) & 
    # 2. 篩選中類為餐飲業
    (df_proc['中類'] == '餐飲業') & 
    # 3. 排除特定名稱 (注意波浪符號的位置)
    (~df_proc['名稱'].isin(['外燴（辦桌）承包', '其他外燴及團膳承包', '團膳承包', '學校營養午餐供應']))
].reset_index(drop=True)
print(f"餐飲業POI篩選完成: {catering_services.shape}  記憶體使用: {catering_services.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# 使用小括號 () 將程式碼包起來，可以進行跨行排版，不需使用反斜線 \
catering_services = (
    catering_services
    .pipe(add_catering_type_column, source_col='名稱')     # 新增型態
    .pipe(enrich_chain_status)                             # 連鎖狀態判斷
    .pipe(split_chain_company_name,                        # 拆分公司名稱
          keyword=["股份有限公司", "有限公司"])
    .pipe(fill_company_and_branch_by_id,                   # 公司/分公司補正
          mapping=company_name,
          id_col="統一編號",
          company_col="公司名稱",
          branch_col="分公司名稱")
)

In [ ]:
catering_services.columns

# 取得座標

In [ ]:
get_coord = pd.read_csv(
    r"C:\labs\geo-grid\data\output_coord\catering_coords.csv", 
    dtype={'統一編號': str}
)

In [ ]:
get_coord[0:20000]

In [ ]:
coord = geocode_tdx_df(
    get_coord[0:20000],
    token=token,
    addr_col="正規化營業地址",     # 要查的地址欄位
    out_addr_col="coords_address", # 回填之標準化地址
    out_lon_col="longitude",       # 回填之經度
    out_lat_col="latitude",        # 回填之緯度
    requests_per_second=10,        # rps，視你的額度調整
    pause_every=45,                # 每45筆暫停
    pause_seconds=60,              # 暫停60秒
    only_fill_na=True,             # 只補缺值
    verbose=True                   # 顯示進度
)

In [ ]:
get_coord = patch_coordinates(
    get_coord,
    coord,
    on_key='統一編號'
)

In [ ]:
get_coord[0:15020]

In [ ]:
get_coord.to_csv(r"C:\labs\geo-grid\data\output_coord\catering_coords.csv", index=False, encoding='utf-8-sig')

In [ ]:
catering_poi = patch_coordinates(
    catering_services,
    get_coord,
    on_key='統一編號'
)

# 餐飲業POI
    - 型別轉換
    - 轉為geometry

In [ ]:
catering_poi_analysis = catering_poi.copy()

In [ ]:
catering_poi_analysis = (
    catering_poi_analysis
    .loc[catering_poi_analysis['longitude'].notna(), [
        '統一編號', '設立日期', '營業人名稱',
        '公司名稱', '分公司名稱', '縣市', '行政區',
        '中類', '名稱', '型態', '營業型態', '連鎖家數',
        'coords_address', 'longitude', 'latitude'
    ]]
    .rename(columns={
        'coords_address': '坐標營業地址',
        'longitude': '經度',
        'latitude': '緯度'
    })
    .assign(
        # 使用 assign 在同一條鏈中直接轉換型態
        # 注意：因為上一步已經 rename 了，所以這裡要用新名稱 '經度' 和 '緯度'
        經度=lambda x: pd.to_numeric(x['經度'], errors='coerce'),
        緯度=lambda x: pd.to_numeric(x['緯度'], errors='coerce')
    )
)

In [ ]:
import geopandas as gpd

# 把一般的 DataFrame 升級成 GeoDataFrame
catering_gdf = gpd.GeoDataFrame(
    catering_poi_analysis, 
    geometry=gpd.points_from_xy(catering_poi_analysis['經度'], catering_poi_analysis['緯度']),
    crs="EPSG:3826"  # 使用台灣經緯度座標系統 (TWD97 / TM2)
)
catering_gdf.info()

In [ ]:
# === 儲存 ===
save_path_gdf = r"C:\labs\geo-grid\data\output_poi\catering_gdf.parquet"
catering_gdf.to_parquet(save_path_gdf)

# # === 讀取 ===
# catering_loaded = gpd.read_parquet(save_path_gdf)